In [8]:
from ipyleaflet import Map, Marker, basemaps, GeoJSON, FullScreenControl, DivIcon, Polyline, CircleMarker
import pandas as pd
import json
from PIL import Image
import networkx as nx

# Discretizing map

In [28]:
stations = pd.read_csv("stations_of_interest.csv") # Load stations of interest

In [27]:
center = (52.198, 5.537)
m = Map(center=center, zoom=9, basemap=basemaps.CartoDB.Positron) # Create map object

with open('provincie_2025.geojson', 'r') as f: # Load outlines
    data = json.load(f)

geo_json = GeoJSON(data=data, style={'fillColor': 'black', 'weight': 0, 'fillOpacity': 1})
m.add(geo_json)

m.add(FullScreenControl())

for _, station in stations.iterrows(): # Add stations with special small markers
    icon = DivIcon(html='<div class="markerr"></div>', center=(station['lat'], station['lng']), draggable=False, bg_pos=(0,0), icon_size=(1,1))
    marker = Marker(location=(station['lat'], station['lng']), icon=icon)
    m.add(marker)

display(m)

Map(center=[52.198, 5.537], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…

Inbetween here is a manual step in photoshop to produce `outline.png`. Here, the area of the Netherlands is filled with green pixels, and stations are marked as red pixels. All other pixels are erased, and the image is cropped.

In [11]:
outline = Image.open("outline.png") # Load photoshopped image
outline.show()

In [26]:
for y in range(outline.height): # Turn pixel values into class labels
    row = []
    for x in range(outline.width):
        r, g, b, a = outline.getpixel((x, y))
        value = (1 if g > 0 else 0) + (2 if r > 0 else 0)
        row.append(value)
    output.append(row)

In [15]:
with open('output.json', 'w+') as o: # Store class labels
    json.dump(output, o)

# Generate figure showing NS stations

In [19]:
nodes = pd.read_csv("nodes.csv") # Import NS data
edges = pd.read_csv("edges.csv")

In [22]:
m2 = Map(center=center, zoom=9, basemap=basemaps.CartoDB.Positron) # Create map object
G = nx.Graph()

for _, row in nodes.iterrows(): # Loop over stations
    G.add_node(row['name'], station=row['station'], lat=row['lat'], lng=row['lng']) # Add station node

    if (stations['code'].eq(row['name'])).any(): # Only add marker if it's a station of interest
        marker = Marker(location=(row['lat'], row['lng']), draggable=False)
        m2.add(marker)

lat = nx.get_node_attributes(G, 'lat')
lng = nx.get_node_attributes(G, 'lng')
for _, row in edges.iterrows(): # Add edges
    G.add_edge(row['from'], row['to'])
    locs = [[lat[row['from']], lng[row['from']]], [lat[row['to']], lng[row['to']]]]
    line = Polyline(locations=locs, color='blue', weight=2)
    m2.add(line)

print(G)
m2.add(FullScreenControl())

display(m2)

Graph with 395 nodes and 434 edges


Map(center=[52.198, 5.537], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_…